In [0]:
%sql
create volume dev.bronze.landing
comment 'This is managed bronze landing volume'

In [0]:
%python
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/04")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/06")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/07")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/08")

In [0]:
# Location for checkpoint

dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/checkpoint/autoloader")

In [0]:
dbutils.fs.ls("databricks-datasets/definitive-guide/data/retail-data/by-day/")

In [0]:
dbutils.fs.cp("databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.cp("databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.cp("databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")
dbutils.fs.cp("databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-05.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")

In [0]:
# Read files using Autoloader with checkpoint
# and schema location "/Volumes/dev/bronze/landing/checkpoint/autoloader"
# File Detection Modes
#  - Directory Listing (uses API calls to detect new files)
#  - File notification (uses Notification and Queue services - requires elevated cloud permissions for setup)

df = (
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("pathGlobFilter", "*.csv")
    .option("header", True)
    .option("cloudFiles.schemaHints", "Quantity int, UnitPrice double")
    .option("cloudFiles.schemaLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1")
    .load("/Volumes/dev/bronze/landing/autoloader_input/*/")
)

In [0]:
import pyspark.sql.functions as F

(
    df
    .withColumn("__file", F.col("_metadata.file_name"))
    .writeStream
    .option("checkpointLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("dev.bronze.invoice_al_1")
)

In [0]:
%sql
select * from dev.bronze.invoice_al_1;

In [0]:
%sql
select __file, count(1) as processed_rows
from dev.bronze.invoice_al_1
group by __file;